<a href="https://colab.research.google.com/github/hanjj753/Market_Prediction/blob/main/Market%20Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Market Prediction 데이터톤용 구조 코드

## 1. 파일 다운로드 및 도움용 코드
- 아래 코드들을 실행해 필요한 파일들 다운로드 및 패키지 설치

In [11]:
#@title Git으로 데이터 가져오기

!git clone https://github.com/hanjj753/Market_Prediction.git

Cloning into 'Market_Prediction'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 73 (delta 22), reused 50 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (73/73), 4.06 MiB | 6.84 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [12]:
#@title 필요한 것들만 남겨두기

!cp /content/Market_Prediction/kaggle_evaluation /content/kaggle_evaluation -r
!cp /content/Market_Prediction/data/train.csv /content/train.csv
!cp /content/Market_Prediction/data/test.csv /content/test.csv
!rm /content/Market_Prediction -r

In [ ]:
#@title import 및 패키지 다운로드

!pip install tqdm polars grpcio google google-api-python-client pyarrow

import os
from pathlib import Path
import datetime
from typing import List

from tqdm import tqdm
from dataclasses import dataclass, asdict

import polars as pl
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet, ElasticNetCV, LinearRegression
from sklearn.preprocessing import StandardScaler

import kaggle_evaluation.default_inference_server

In [7]:
#@title Score 계산 함수
import pandas.api.types

MIN_INVESTMENT = 0
MAX_INVESTMENT = 2


class ParticipantVisibleError(Exception):
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """
    Calculates a custom evaluation metric (volatility-adjusted Sharpe ratio).

    This metric penalizes strategies that take on significantly more volatility
    than the underlying market.

    Returns:
        float: The calculated adjusted Sharpe ratio.
    """

    if not pandas.api.types.is_numeric_dtype(submission['prediction']):
        raise ParticipantVisibleError('Predictions must be numeric')

    solution = solution
    solution['position'] = submission['prediction']

    if solution['position'].max() > MAX_INVESTMENT:
        raise ParticipantVisibleError(f'Position of {solution["position"].max()} exceeds maximum of {MAX_INVESTMENT}')
    if solution['position'].min() < MIN_INVESTMENT:
        raise ParticipantVisibleError(f'Position of {solution["position"].min()} below minimum of {MIN_INVESTMENT}')

    solution['strategy_returns'] = solution['risk_free_rate'] * (1 - solution['position']) + solution['position'] * solution['forward_returns']

    # Calculate strategy's Sharpe ratio
    strategy_excess_returns = solution['strategy_returns'] - solution['risk_free_rate']
    strategy_excess_cumulative = (1 + strategy_excess_returns).prod()
    strategy_mean_excess_return = (strategy_excess_cumulative) ** (1 / len(solution)) - 1
    strategy_std = solution['strategy_returns'].std()

    trading_days_per_yr = 252
    if strategy_std == 0:
        raise ParticipantVisibleError('Division by zero, strategy std is zero')
    sharpe = strategy_mean_excess_return / strategy_std * np.sqrt(trading_days_per_yr)
    strategy_volatility = float(strategy_std * np.sqrt(trading_days_per_yr) * 100)

    # Calculate market return and volatility
    market_excess_returns = solution['forward_returns'] - solution['risk_free_rate']
    market_excess_cumulative = (1 + market_excess_returns).prod()
    market_mean_excess_return = (market_excess_cumulative) ** (1 / len(solution)) - 1
    market_std = solution['forward_returns'].std()

    market_volatility = float(market_std * np.sqrt(trading_days_per_yr) * 100)

    if market_volatility == 0:
        raise ParticipantVisibleError('Division by zero, market std is zero')

    # Calculate the volatility penalty
    excess_vol = max(0, strategy_volatility / market_volatility - 1.2) if market_volatility > 0 else 0
    vol_penalty = 1 + excess_vol

    # Calculate the return penalty
    return_gap = max(
        0,
        (market_mean_excess_return - strategy_mean_excess_return) * 100 * trading_days_per_yr,
    )
    return_penalty = 1 + (return_gap**2) / 100

    # Adjust the Sharpe ratio by the volatility and return penalty
    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    return min(float(adjusted_sharpe), 1_000_000)

## 2. 참고용 예시 코드
- I1~I9 데이터만을 이용해 Linear Regression
- market_forward_excess_returns 을 종속변수로 잡아 훈련시켜 predict를 한 뒤에, np.clip 함수를 이용해 0~2 사이의 값으로 변환.
- np.clip(값, 0, 2)은 단순히 2 이상의 값이면 2, 0 이하의 값이면 0으로 바꿔주는 함수.

In [18]:
DATA_PATH = Path('')
TARGET_COLUMN = 'market_forward_excess_returns'
REMOVE_COLUMNS = ['date_id', 'forward_returns','risk_free_rate','market_forward_excess_returns']

SELECTED_FEATURES = ['I1','I2','I3','I4','I5','I6','I7','I8','I9']

MIN_SIGNAL: float = 0.0
MAX_SIGNAL: float = 2.0
SIGNAL_MULTIPLIER: float = 400.0

In [19]:
df_train = pd.read_csv(DATA_PATH / "train.csv").iloc[:-180]
df_test = pd.read_csv(DATA_PATH / "test.csv")

X_train = df_train.dropna()[SELECTED_FEATURES]
y_train = df_train.dropna()[TARGET_COLUMN]

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [20]:
def predict(test: pl.DataFrame) -> float:

    if model is None:
        return 0.0

    try:
        test_pd = test.to_pandas()
        X_test = test_pd[SELECTED_FEATURES]
        X_test = X_test.fillna(0)
        predicted_return_array = model.predict(X_test)
        predicted_return = float(predicted_return_array[0])

        signal = np.clip(
            predicted_return * SIGNAL_MULTIPLIER + 1,
            MIN_SIGNAL,
            MAX_SIGNAL
        )
        return float(signal)

    except Exception as e:
        return 0.0

inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/content/',))

## 3. 실제 사용할 코드

In [14]:
def predict(test: pl.DataFrame) -> float:
    """Replace this function with your inference code.
    You can return either a Pandas or Polars dataframe, though Polars is recommended for performance.
    Each batch of predictions (except the very first) must be returned within 5 minutes of the batch features being provided.
    """
    return 0.0


# When your notebook is run on the hidden test set, inference_server.serve must be called within 15 minutes of the notebook starting
# or the gateway will throw an error. If you need more than 15 minutes to load your model you can do so during the very
# first `predict` call, which does not have the usual 1 minute response deadline.
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/content/',))

In [15]:
#@title Prediction이 잘 되었는지 확인

submission_df = pd.read_parquet('submission.parquet')
print(submission_df.head())

   date_id  prediction
0     8810         0.0
1     8811         0.0
2     8812         0.0
3     8813         0.0
4     8814         0.0


In [21]:
#@title 점수 계산

try:
    solution_df = (
        pl.read_csv("train.csv")
        .tail(180)
        .to_pandas() # score 함수는 pandas DataFrame을 사용
    )

    submission_df = pd.read_parquet("submission.parquet")

    local_score = score(
        solution=solution_df,
        submission=submission_df,
        row_id_column_name="batch_id"
    )

    print("---" * 10)
    print(f"(정답: {len(solution_df)}, 예측: {len(submission_df)})")
    print(f"Score(Adjusted Sharpe Ratio): {local_score:.5f}")
    print("---" * 10)

except Exception as e:
    print(e)

------------------------------
(정답: 180, 예측: 180)
Score(Adjusted Sharpe Ratio): 0.51311
------------------------------


## 4. 코드를 저장하고 싶으면 [파일] - [다운로드] - [.ipynb 다운로드]